In [1]:
import sys
import cv2
import mediapipe as mp
import glob
import numpy as np
import json
import os
import time
from os.path import exists

In [2]:
datapath = '/mnt/HDD04/Gallaudet_exp1_fusion_data/videos/'
vid_savepath = '/mnt/HDD04/ICASSP_2023/Data/annotated_videos/'
annot_path = '/mnt/HDD04/ICASSP_2023/Data/annotated_images/'
skelpath = '/mnt/HDD04/ICASSP_2023/Data/skeletons/'
frame_path = skelpath.replace('skeletons', 'images')

In [3]:
files = glob.glob(datapath + '*avi')
print('Num files present:', len(files))
print(files[0])

Num files present: 2244
/mnt/HDD04/Gallaudet_exp1_fusion_data/videos/2022_04_13_14_14_53_subj7_Exp1_class23_part2_word112_cnt3.avi


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [7]:
f = files[0]
fname = f.split('/')[-1]
datapath + fname

'/mnt/HDD04/Gallaudet_exp1_fusion_data/videos/2022_04_13_14_14_53_subj7_Exp1_class23_part2_word112_cnt3.avi'

In [11]:
# extract skeleton
start_time = time.time()
cnt = 0
for f in files:
    fname = f.split('/')[-1]

    cnt += 1
    now = time.time()
    print('Processing file ' + str(cnt) + '/' + str(len(files)))
    cap = cv2.VideoCapture(datapath + fname)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = 0
    end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    subfolder_frame = frame_path + fname[:-4]
          
    if not exists(subfolder_frame):
        os.mkdir(subfolder_frame)
        # extract frames
#             for frame_no in range(start_frame, end_frame + 1):
        for frame_no in range(start_frame, end_frame + 1):
            cap.set(1,frame_no)  # Where frame_no is the frame you want
            ret, frame = cap.read()  # Read the frame
#                     frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            cv2.imwrite(subfolder_frame + '/' + fname[:-4] + '_frame' + f'{frame_no:05d}' + '.jpg', frame)

    # MediaPipe
    subfolder_skel = skelpath + fname.split('/')[-1][:-4]
    if not exists(subfolder_skel):
        os.mkdir(subfolder_skel)
        IMAGE_FILES = sorted(glob.glob(subfolder_frame + '/*jpg'))
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:

            image = cv2.flip(cv2.imread(IMAGE_FILES[0]), 1)
            fname2 = IMAGE_FILES[0].split('/')[-1]
            prev_results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            for idx, file in enumerate(IMAGE_FILES):
                # Read an image, flip it around y-axis for correct handedness output (see
                # above).
                image = cv2.flip(cv2.imread(file), 1)
                fname2 = file.split('/')[-1]
                # Convert the BGR image to RGB before processing.
                results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                # Print handedness and draw hand landmarks on the image.
#                     print('Frame: ', idx, '/', len(IMAGE_FILES))
                # print('Handedness:', results.multi_handedness)
                if not results.multi_hand_landmarks:
                    if not prev_results.multi_hand_landmarks:
                        continue
                    else:
                        results = prev_results  # to prevent frame elimination due to non-existent hands
                prev_results = results
                image_height, image_width, _ = image.shape
                annotated_image = image.copy()
                data = []

                for hand_id, hand_landmarks in enumerate(results.multi_hand_landmarks):


                  # print('hand_landmarks:', hand_landmarks)
                  # print(
                  #     f'Index finger tip coordinates: (',
                  #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                  #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                  # )
                  # print(dir(hand_landmarks.landmark[0]))
                  # print(hand_landmarks.landmark[0])
                  # print(hand_landmarks.landmark[0].x)

                    for land_id, mark in enumerate(hand_landmarks.landmark):

                        if land_id == 0:
                            x_temp = [mark.x]
                            y_temp = [mark.y]
                            z_temp = [mark.z]
                        else:
                            x_temp = np.concatenate((x_temp, [mark.x]), 0)
                            y_temp = np.concatenate((y_temp, [mark.y]), 0)
                            z_temp = np.concatenate((z_temp, [mark.z]), 0)

                    data.append(x_temp)
                    data.append(y_temp)
                    data.append(z_temp)
                    mp_drawing.draw_landmarks(
                          annotated_image,
                          hand_landmarks,
                          mp_hands.HAND_CONNECTIONS,
                          mp_drawing_styles.get_default_hand_landmarks_style(),
                          mp_drawing_styles.get_default_hand_connections_style())


                with open(subfolder_skel + '/' + fname2.replace('jpg', 'txt'), 'w') as f:
                    for elem in data:
                        for e in elem:
                            f.write(str(e) + ' ')
                        f.write('\n')
                subfolder_im = annot_path + fname[:-4]
                if not exists(subfolder_im):
                    os.mkdir(subfolder_im)
                cv2.imwrite(subfolder_im + '/' + fname2.replace('jpg', 'png'), cv2.flip(annotated_image, 1))

            # create video
#                     files = sorted(glob.glob(subfolder_im + '/*png'))
#                     frame = cv2.imread(files[0])
#                     height, width, layers = frame.shape
#                     video_name = vid_savepath + fname.split('/')[-1][:-4] + '.avi'
#                     video = cv2.VideoWriter(video_name, 0, fps, (width, height))
#                     for image in files:
#                         video.write(cv2.imread(image))
#                     video.release()

    print('Elapsed time:', time.time() - now, 'sec')
    print('Num. processed file:' + str(cnt) + '/' + str(len(files)))
    print('Total Elapsed time:', (time.time() - start_time)/60, ' min')

#             if j == 4:
#                 break
#             break

#     break


Processing file 1/2244
Elapsed time: 10.431488037109375 sec
Num. processed file:1/2244
Total Elapsed time: 0.17389235893885294  min
Processing file 2/2244
Elapsed time: 10.289266347885132 sec
Num. processed file:2/2244
Total Elapsed time: 0.34542959928512573  min
Processing file 3/2244
Elapsed time: 8.822490453720093 sec
Num. processed file:3/2244
Total Elapsed time: 0.492484458287557  min
Processing file 4/2244
Elapsed time: 10.879159212112427 sec
Num. processed file:4/2244
Total Elapsed time: 0.6738417506217956  min
Processing file 5/2244
Elapsed time: 8.536614656448364 sec
Num. processed file:5/2244
Total Elapsed time: 0.8161372741063436  min
Processing file 6/2244
Elapsed time: 7.306500434875488 sec
Num. processed file:6/2244
Total Elapsed time: 0.9379207253456116  min
Processing file 7/2244
Elapsed time: 12.708387613296509 sec
Num. processed file:7/2244
Total Elapsed time: 1.1497703552246095  min
Processing file 8/2244
Elapsed time: 8.197332620620728 sec
Num. processed file:8/2244

Elapsed time: 12.446141004562378 sec
Num. processed file:64/2244
Total Elapsed time: 10.217370212078094  min
Processing file 65/2244
Elapsed time: 7.860021352767944 sec
Num. processed file:65/2244
Total Elapsed time: 10.348386573791505  min
Processing file 66/2244
Elapsed time: 8.786137819290161 sec
Num. processed file:66/2244
Total Elapsed time: 10.494871242841084  min
Processing file 67/2244
Elapsed time: 9.077896118164062 sec
Num. processed file:67/2244
Total Elapsed time: 10.646195503075917  min
Processing file 68/2244
Elapsed time: 7.785080194473267 sec
Num. processed file:68/2244
Total Elapsed time: 10.775960059960683  min
Processing file 69/2244
Elapsed time: 9.603167057037354 sec
Num. processed file:69/2244
Total Elapsed time: 10.936018176873525  min
Processing file 70/2244
Elapsed time: 7.520908832550049 sec
Num. processed file:70/2244
Total Elapsed time: 11.061389025052389  min
Processing file 71/2244
Elapsed time: 8.791634559631348 sec
Num. processed file:71/2244
Total Elaps

Elapsed time: 9.237367391586304 sec
Num. processed file:126/2244
Total Elapsed time: 19.93239827156067  min
Processing file 127/2244
Elapsed time: 11.47232174873352 sec
Num. processed file:127/2244
Total Elapsed time: 20.123619894186657  min
Processing file 128/2244
Elapsed time: 7.818066358566284 sec
Num. processed file:128/2244
Total Elapsed time: 20.253975053628285  min
Processing file 129/2244
Elapsed time: 8.751667022705078 sec
Num. processed file:129/2244
Total Elapsed time: 20.39985323746999  min
Processing file 130/2244
Elapsed time: 7.236290454864502 sec
Num. processed file:130/2244
Total Elapsed time: 20.520474604765575  min
Processing file 131/2244
Elapsed time: 8.158942222595215 sec
Num. processed file:131/2244
Total Elapsed time: 20.656469055016835  min
Processing file 132/2244
Elapsed time: 8.39655065536499 sec
Num. processed file:132/2244
Total Elapsed time: 20.796448191006977  min
Processing file 133/2244
Elapsed time: 9.456852912902832 sec
Num. processed file:133/2244


Elapsed time: 11.113500118255615 sec
Num. processed file:188/2244
Total Elapsed time: 29.932520683606466  min
Processing file 189/2244
Elapsed time: 10.17078709602356 sec
Num. processed file:189/2244
Total Elapsed time: 30.102051766713462  min
Processing file 190/2244
Elapsed time: 10.163424253463745 sec
Num. processed file:190/2244
Total Elapsed time: 30.271478044986726  min
Processing file 191/2244
Elapsed time: 12.436490535736084 sec
Num. processed file:191/2244
Total Elapsed time: 30.47877926826477  min
Processing file 192/2244
Elapsed time: 12.024158477783203 sec
Num. processed file:192/2244
Total Elapsed time: 30.679222559928895  min
Processing file 193/2244
Elapsed time: 13.207891941070557 sec
Num. processed file:193/2244
Total Elapsed time: 30.89938268661499  min
Processing file 194/2244
Elapsed time: 12.501803398132324 sec
Num. processed file:194/2244
Total Elapsed time: 31.107759221394858  min
Processing file 195/2244
Elapsed time: 9.106218576431274 sec
Num. processed file:19

Elapsed time: 8.128004550933838 sec
Num. processed file:250/2244
Total Elapsed time: 40.36653732061386  min
Processing file 251/2244
Elapsed time: 8.382499933242798 sec
Num. processed file:251/2244
Total Elapsed time: 40.5062566836675  min
Processing file 252/2244
Elapsed time: 14.526894330978394 sec
Num. processed file:252/2244
Total Elapsed time: 40.748389879862465  min
Processing file 253/2244
Elapsed time: 9.705102920532227 sec
Num. processed file:253/2244
Total Elapsed time: 40.91014941930771  min
Processing file 254/2244
Elapsed time: 11.881539583206177 sec
Num. processed file:254/2244
Total Elapsed time: 41.108202477296196  min
Processing file 255/2244
Elapsed time: 8.11459732055664 sec
Num. processed file:255/2244
Total Elapsed time: 41.243483189741774  min
Processing file 256/2244
Elapsed time: 8.561971664428711 sec
Num. processed file:256/2244
Total Elapsed time: 41.38619366089503  min
Processing file 257/2244
Elapsed time: 13.924533367156982 sec
Num. processed file:257/2244


...

Elapsed time: 5.807729244232178 sec
Num. processed file:1970/2244
Total Elapsed time: 324.8700986901919  min
Processing file 1971/2244
Elapsed time: 7.784665107727051 sec
Num. processed file:1971/2244
Total Elapsed time: 324.99985296726226  min
Processing file 1972/2244
Elapsed time: 8.49384880065918 sec
Num. processed file:1972/2244
Total Elapsed time: 325.1414296309153  min
Processing file 1973/2244
Elapsed time: 7.929292440414429 sec
Num. processed file:1973/2244
Total Elapsed time: 325.2736072063446  min
Processing file 1974/2244
Elapsed time: 9.888453722000122 sec
Num. processed file:1974/2244
Total Elapsed time: 325.438424440225  min
Processing file 1975/2244
Elapsed time: 8.665089845657349 sec
Num. processed file:1975/2244
Total Elapsed time: 325.58285341660184  min
Processing file 1976/2244
Elapsed time: 4.365505695343018 sec
Num. processed file:1976/2244
Total Elapsed time: 325.6556400299072  min
Processing file 1977/2244
Elapsed time: 9.722641468048096 sec
Num. processed file

Elapsed time: 8.503397464752197 sec
Num. processed file:2031/2244
Total Elapsed time: 334.51690630118054  min
Processing file 2032/2244
Elapsed time: 8.519068002700806 sec
Num. processed file:2032/2244
Total Elapsed time: 334.65890497366587  min
Processing file 2033/2244
Elapsed time: 11.942734479904175 sec
Num. processed file:2033/2244
Total Elapsed time: 334.85795992215475  min
Processing file 2034/2244
Elapsed time: 9.29647159576416 sec
Num. processed file:2034/2244
Total Elapsed time: 335.012913608551  min
Processing file 2035/2244
Elapsed time: 9.753881931304932 sec
Num. processed file:2035/2244
Total Elapsed time: 335.1754899342855  min
Processing file 2036/2244
Elapsed time: 9.553039789199829 sec
Num. processed file:2036/2244
Total Elapsed time: 335.33471718629204  min
Processing file 2037/2244
Elapsed time: 12.103404760360718 sec
Num. processed file:2037/2244
Total Elapsed time: 335.53645680745444  min
Processing file 2038/2244
Elapsed time: 9.157157182693481 sec
Num. processed

Elapsed time: 12.920839071273804 sec
Num. processed file:2092/2244
Total Elapsed time: 344.6576925794283  min
Processing file 2093/2244
Elapsed time: 10.80616545677185 sec
Num. processed file:2093/2244
Total Elapsed time: 344.8378062327703  min
Processing file 2094/2244
Elapsed time: 13.125919103622437 sec
Num. processed file:2094/2244
Total Elapsed time: 345.05661394198734  min
Processing file 2095/2244
Elapsed time: 8.752519130706787 sec
Num. processed file:2095/2244
Total Elapsed time: 345.2025085528692  min
Processing file 2096/2244
Elapsed time: 8.29985499382019 sec
Num. processed file:2096/2244
Total Elapsed time: 345.34084444443386  min
Processing file 2097/2244
Elapsed time: 8.128563165664673 sec
Num. processed file:2097/2244
Total Elapsed time: 345.47633053859073  min
Processing file 2098/2244
Elapsed time: 7.554409027099609 sec
Num. processed file:2098/2244
Total Elapsed time: 345.60226377646126  min
Processing file 2099/2244
Elapsed time: 10.959727764129639 sec
Num. processe

Elapsed time: 10.04910945892334 sec
Num. processed file:2153/2244
Total Elapsed time: 354.69340102275214  min
Processing file 2154/2244
Elapsed time: 12.402228832244873 sec
Num. processed file:2154/2244
Total Elapsed time: 354.9001130501429  min
Processing file 2155/2244
Elapsed time: 8.01388168334961 sec
Num. processed file:2155/2244
Total Elapsed time: 355.0336963852247  min
Processing file 2156/2244
Elapsed time: 8.982358455657959 sec
Num. processed file:2156/2244
Total Elapsed time: 355.1834084590276  min
Processing file 2157/2244
Elapsed time: 14.570940017700195 sec
Num. processed file:2157/2244
Total Elapsed time: 355.42627235651014  min
Processing file 2158/2244
Elapsed time: 8.379792213439941 sec
Num. processed file:2158/2244
Total Elapsed time: 355.5659436027209  min
Processing file 2159/2244
Elapsed time: 7.565189599990845 sec
Num. processed file:2159/2244
Total Elapsed time: 355.69206748406094  min
Processing file 2160/2244
Elapsed time: 8.697417736053467 sec
Num. processed 

Elapsed time: 10.969773530960083 sec
Num. processed file:2214/2244
Total Elapsed time: 364.83201695283253  min
Processing file 2215/2244
Elapsed time: 11.658372640609741 sec
Num. processed file:2215/2244
Total Elapsed time: 365.0263321121534  min
Processing file 2216/2244
Elapsed time: 7.65750789642334 sec
Num. processed file:2216/2244
Total Elapsed time: 365.1539754708608  min
Processing file 2217/2244
Elapsed time: 13.643141269683838 sec
Num. processed file:2217/2244
Total Elapsed time: 365.3813736995061  min
Processing file 2218/2244
Elapsed time: 10.090064287185669 sec
Num. processed file:2218/2244
Total Elapsed time: 365.54957287311555  min
Processing file 2219/2244
Elapsed time: 8.581031560897827 sec
Num. processed file:2219/2244
Total Elapsed time: 365.69259817202885  min
Processing file 2220/2244
Elapsed time: 12.512291193008423 sec
Num. processed file:2220/2244
Total Elapsed time: 365.9011587460836  min
Processing file 2221/2244
Elapsed time: 13.631168127059937 sec
Num. proces